# The Eye Of The Typer Dataset

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from typing import Literal
from datetime import datetime, timedelta
from pprint import pprint

import polars as pl
import rerun as rr
import cv2 as cv

from the_eye_of_the_typer import *

In [ ]:
# %env set THE_EYE_OF_THE_TYPER_DATASET_PATH
df = read_participant_characteristics()
df

In [ ]:
participants = [Participant.create(**row) for row in df.iter_rows(named=True)]

print("Participants:", len(participants), end="\n\n")
pprint(participants[0])

In [ ]:
p = participants[0]

In [ ]:
rr.init("EOTT", recording_id=p.participant_id, spawn=True)
rr.log(
    "participant",
    rr.TextDocument(
        "\n".join(f"[{key}]\n{value}\n" for key, value in p.to_dict().items())
    ),
    timeless=True,
)

screen_cap: cv.VideoCapture | None = None
webcam_cap: cv.VideoCapture | None = None

screen_width = p.display_width
screen_height = p.display_height

if p.screen_recording is not None:
    screen_cap = cv.VideoCapture(str(p.screen_recording_path))
    screen_scale_factor = 2
    screen_width = screen_width // screen_scale_factor
    screen_height = screen_height // screen_scale_factor

rr.log(
    f"screen",
    rr.Boxes2D(
        sizes=[[screen_width, screen_height]],
        centers=[[screen_width / 2, screen_height / 2]],
    ),
    timeless=True,
)

rr.log(
    "participant/pupil/left/tobii",
    rr.SeriesLine(color=[255, 255, 0], width=1, name="left pupil diameter (tobii)"),
    timeless=True,
)
rr.log(
    "participant/pupil/right/tobii",
    rr.SeriesLine(color=[255, 0, 255], width=1, name="right pupil diameter (tobii)"),
    timeless=True,
)

timeline_df = get_timeline(p)

frame_index: int
source_name: Literal["tobii", "log", "screen", "webcam"]
offset_time: timedelta
study_name: str | None
study_index: int | None
for (
    frame_index,
    offset_time,
    source_name,
    study_name,
    study_index,
) in timeline_df.iter_rows():
    rr.set_time_sequence(f"{source_name}_index", frame_index)
    rr.set_time_seconds("capture_time", offset_time.total_seconds())

    match source_name:
        case "screen" if screen_cap is not None:
            rr.set_time_seconds(
                "screen_time", screen_cap.get(cv.CAP_PROP_POS_MSEC) / 1_000
            )

            assert p.screen_recording is not None
            assert p.screen_offset is not None

            success, frame = screen_cap.read()

            if not success or frame_index % 4 != 0:
                continue

            frame = cv.resize(frame, (screen_width, screen_height))
            frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

            rr.log("screen", rr.Image(frame))

        case "tobii":
            entry: TobiiEntry = p.tobii_gaze_predictions.row(frame_index, named=True)

            for eye in ("left", "right"):
                if entry[f"{eye}_gaze_point_validity"]:
                    x, y = entry[f"{eye}_gaze_point_on_display_area"]
                    rr.log(
                        f"screen/gazepoint/{eye}/tobii",
                        rr.Points2D(
                            [[x * screen_width, y * screen_height]],
                            colors=[[0, 0, 255]],
                            radii=[1],
                        ),
                    )
                else:
                    rr.log(
                        f"screen/gazepoint/{eye}/tobii",
                        rr.Clear(recursive=True),
                    )

                if (
                    entry[f"{eye}_pupil_validity"]
                    and entry[f"{eye}_pupil_diameter"] > 0
                ):
                    rr.log(
                        f"participant/pupil/{eye}/tobii",
                        rr.Scalar(entry[f"{eye}_pupil_diameter"]),
                    )
                else:
                    rr.log(f"participant/pupil/{eye}/tobii", rr.Clear(recursive=True))

        case "log":
            level_map = {
                "start": rr.TextLogLevel.CRITICAL,
                "stop": rr.TextLogLevel.ERROR,
                "mouse": rr.TextLogLevel.TRACE,
                "scroll": rr.TextLogLevel.DEBUG,
                "click": rr.TextLogLevel.WARN,
                "text": rr.TextLogLevel.INFO,
            }
            entry: LogEntry = p.user_interaction_logs.row(frame_index, named=True)
            event_type = entry["event"]
            if event_type is not None:
                rr.log(
                    "participant/event",
                    rr.TextLog(entry["event"], level=level_map[entry["event"]]),
                )

            match event_type:
                case "mouse":
                    rr.log(
                        "screen/mouse",
                        rr.Points2D(
                            [
                                [
                                    entry["screen_x"] / screen_scale_factor,
                                    entry["screen_y"] / screen_scale_factor,
                                ]
                            ],
                            colors=[(255, 255, 0)],
                            radii=[1],
                        ),
                    )

# end of logging
if screen_cap is not None:
    screen_cap.release()

if webcam_cap is not None:
    webcam_cap.release()